# Code for penetrable Helmholtz problem
The following problem is solved for a bounded inghomogeneity
$$
\Delta u^s + \kappa^2 n(x) u^s=-\kappa^2(n(x)-1)u^i \mbox{ in }\mathbb{R}^2
$$
where $u^i$ is the known incident field.  We must have $n(x)=1$ for $|x|>R$ and some $R>0$ (this is called ``air'' in the code).<br>

The domain is truncated using an annular PML centered at the origin starting at a radius *pml_rad* set by the user.  The PML has thickness *pml_delta* also set by the user.  Mor on this below.

### Import netgen and the geometries needed.  

The library system is used to stored results. The file *geometries_penetrable.py* has server different scatterers defined that we have used for our studies.

In [ ]:
# forward problem setup for U-shaped scatter or pair of dielectric circles only
import sys         
from ngsolve.webgui import Draw # for draw in jupyter
from netgen.geom2d import SplineGeometry
from ngsolve import *
from geometries_penetrable import circle,square,peanut,ellipses,Ushape
import numpy as np

### Main functions

This is the netgen code used to solve the problem.  It should not need to be changed.  The function *farf2d* computes the far field pattern from the near field computed by FEM.  The function *helmsol* sets up the bilineaer form and solves the near field problem.

In [ ]:
def farf2d(u,mesh,farp,kappa,porder):
    # Input
    # u - the near field solution computed by helmsol
    # mesh - the NGSpy mesh
    # farp - parameters for the far field pattern 
    # kappa - the wavenumber
    # porder - the order of the polynmials for the FE calculation
    #
    # Output
    # uinf - the far field pattern
    # phi - angles for directions where uinf is calculated 
    nphi=farp["n"]
    cphi=farp["cent"]
    appphi=farp["app"]
    nv=specialcf.normal(mesh.dim)
    phi=np.zeros(nphi)
    uinf=np.zeros(nphi,dtype=complex)
    fesa=H1(mesh, order=porder, complex=True, definedon=mesh.Materials("air"))
    for jp in range(0,nphi):
        phi[jp]=(cphi-appphi/2)+appphi*jp/(nphi-1)
        xhat=(np.cos(phi[jp]),np.sin(phi[jp]))
        Eout = exp(-1J*kappa*(x*xhat[0]+y*xhat[1]))
        func1=CoefficientFunction(-1j*kappa*(CoefficientFunction(xhat)*nv)*Eout * u)
        uinf1=Integrate(tuple(func1),mesh,order=porder+1,definedon=
                            mesh.Boundaries("scatterer"))
        vv=GridFunction(fesa)
        vv.vec[:]=0
        vv.Set(Eout,BND,definedon=mesh.Boundaries("scatterer"))
        fvv=CoefficientFunction(grad(vv)*grad(u)-kappa*kappa*vv*u)
        uinf2=Integrate(tuple(fvv),mesh,order=porder+1,definedon=mesh.Materials("air"))
        uinf[jp]=exp(1J*np.pi/4)/np.sqrt(8*np.pi*kappa)*(uinf1+uinf2)
    return(uinf,phi)



def helmsol(mesh,porder,ncoef,kappa,incp,farp):
    # Input
    # mesh - the NGSpy mesh
    # porder - order of the FE space
    # ncoef - a coefficient function that gives the value of *n* at points in the mesh (in the PML n=1)
    # kappa - wavenunmber
    # incp - parameters for the incident field
    # farp - parameters for the far field
    #
    # Returns
    # uinf - far field matrix
    # phi - measurement angles
    # theta - incident angles
    #
    fes = H1(mesh, order=porder, complex=True)
    u = fes.TrialFunction()
    v = fes.TestFunction()
    a = BilinearForm(fes)
    a += SymbolicBFI(grad(u)*grad(v) - kappa**2*ncoef*u*v)
    a += SymbolicBFI(-1j*kappa*u*v,definedon=mesh.Boundaries("outerbnd"))
    print('Number of DoFs: ',fes.ndof)
    gfu = GridFunction(fes)
    Draw(gfu,mesh,'us')
    with TaskManager():
        a.Assemble()
        Ainv=a.mat.Inverse()   
    uinf=np.zeros((farp["n"],incp["n"]),dtype=complex)
    theta=np.zeros(incp["n"]);
    center=incp["cent"]
    app=incp["app"]
    for ip in range(0,incp["n"]):
        if ip%10==0:
            print("Done ip = ", ip,' of ',incp["n"])
        if incp["n"]==1:
            theta[0]=0.0
        else:
            theta[ip]=(center-app/2)+app*ip/(incp["n"]-1)
        d=[np.cos(theta[ip]),np.sin(theta[ip])]
        with TaskManager():
            b = LinearForm(fes)
            ui=exp(1J*kappa*(d[0]*x+d[1]*y))
            b += SymbolicLFI(kappa*kappa*(ncoef-1)*ui * v)
            b.Assemble()
            gfu.vec.data =  Ainv * b.vec
            Redraw()
        uinf[:,ip],phi=farf2d(gfu,mesh,farp,kappa,porder)
    return(uinf,theta,phi)

## Main code with comments
This script sets the necessary parameters, creates the mesh and solves the problem.  The far field data is stored in a file *farf.npz*.<br>

Steps:
1. Choose polynomial order and incident and measurement sectors (the sector is usually $[0,2\pi]$ in which case the measurement for $\theta=0$ and $\theta=2\pi$ are repeated (same for $\phi$).
2. Define parameters for the mesh.  These are needed to set correct sizes for the domain and triangles.
3. Set parameters for the mesh size (my choice is heuristic and needs to be checked).
4. Set the *pml_radius* and create the mesh.  The parameter *pml_radius* needs to be set to provide an air layer between the outermost scatterer and the PML.  I use the a rough approximation of the radius of the smallest circle conating the scatterers plus at least one wavelength.
5. Set the CoefficientFunction for the n(x)
6. Solve the problem and output the results

In [ ]:
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    porder=4 # order of the polynomials in the FEM
    
    # both dictionaries have the same format
    #  incp["n"] = number of angles equally spaced in the aperture including both end points
    #  incp["app"] = angular extent of the aperture
    #  incp["cent"] = angle at the middle of the aperture
    inc_p={"n":101,"app":2*np.pi,"cent":0} # parameters for incident wave
    far_p={"n":101,"app":inc_p["app"],"cent":0} # params for measurement
    #
    geom="square"  # set the geometry here (make sure it is loaded in the firs section)
    if geom=="ellipses":
        kappa=10 # Wavenumber
        numellip=2 # number of ellipses
        nvals=[1.5,1.8] # n for each ellipse
        nval_scale=np.max(nvals) # This variable is to choose a scaling for the mesh
        ellipses_data={'numellip':numellip,
                    'R1a':0.1,'R1b':.2,'xcen1':(0.,0.),'ang1':0,'ind1':3,
                    'R2a':0.3,'R2b':.1,'xcen2':(0,0.5),'ang2':0,'ind2':4}
    elif geom=="Ushape":
        kappa=10 # Wavenumber
        nval = 2 # n inside scatterer for Ushape
        nval_scale=nval # This variable is to choose a scaling for the mesh
        Box=[-.6,.6,-.4,.4] # xmin,xmax,ymin,ymax for U Box determines the overall size of U
        thick=0.2 # thickness of arms
    elif geom=="circle":
        kappa=10
        nval=2
        nval_scale=2
        R=1 # radius of circle
    elif geom=="square":
        kappa=6
        nval=2.5
        nval_scale=nval
    elif geom=="peanut": # use with care, the reults look odd!
        kappa=8
        nval=3
        nval_scale=nval
    else:
        print("Implement this geometry",geom)
        zzz # force a stop
    # meshing parameters.  These are heuristic and may need changing.  Perform a mesh refinement study.
    hmax_s=2*np.pi/kappa/np.sqrt(nval_scale)/8 # mesh size in scatterer
    hmax_a=2*np.pi/kappa/8 # mesh size in air
    print('hmax_s=',hmax_s)
    print('hmax_a=',hmax_a)
    print('wavelength lambda=',2*np.pi/kappa)
    # basic PML parameters
    pml_delta=2*np.pi/kappa  # thickness of the PML is one wavelength
    print('PML width= ',pml_delta)
    pml_parameter=1j
    print('PML alpha =',pml_parameter)
    # set the inner radius of the PML.  See meshing.pdf for comments
    # on how to create a new figure.
    if geom=="circle":
        pml_rad=R+2*2*np.pi/kappa  # radius of inner edge of the PML
        mesh=circle(hmax_s=hmax_s,hmax_a=hmax_a,pml_rad=pml_rad,
                            pml_delta=pml_delta,order=porder,R=R)
    elif geom=="square":
        pml_rad=1+2*2*np.pi/kappa  # radius of inner edge of the PML
        mesh=square(hmax_s=hmax_s,hmax_a=hmax_a,pml_rad=pml_rad,
                            pml_delta=pml_delta,order=porder)
    elif geom=="ellipses":
        pml_rad=1+2*2*np.pi/kappa  # radius of inner edge of the PML        
        mesh=ellipses(hmax_a=hmax_a,hmax_s=hmax_s, order=porder,pml_rad=pml_rad,
                    pml_delta=pml_delta,ellip_data=ellipses_data)
    elif geom=="Ushape":
        pml_rad=1+2*2*np.pi/kappa  # radius of inner edge of the PML
        mesh=Ushape(hmax_a=hmax_a,hmax_s=hmax_s,pml_rad=pml_rad,
            pml_width=pml_delta,Box=Box,t=thick)
    elif geom=="peanut":
        pml_rad=2.4+2*2*np.pi/kappa  # radius of inner edge of the PML
        mesh=peanut(hmax_s=hmax_s,hmax_a=hmax_a,pml_rad=pml_rad,
                            pml_delta=pml_delta,order=porder)
    else:
        print("implement geometr",geometry)
        zzz
    print(mesh.GetMaterials())
    print(mesh.GetBoundaries())
    mesh.SetPML(pml.Radial(rad=pml_rad,alpha=pml_parameter,origin=(0,0)),
                    "pmlregion")
    if geom=="ellipses":
        ncoef=CoefficientFunction([nvals[0] if mat=="ellip1" else nvals[1] if mat=="ellip2" else nvals[2] if mat=="ellip3"  
                                   else 1  for mat in mesh.GetMaterials()])
        scatterer=CoefficientFunction([5 if mat=="ellip3" else 4 if mat=="ellip2"
                else 3 if mat=="ellip1" else 1 if mat=="air" else 2
                                for mat in mesh.GetMaterials()])
    elif geom=="square" or geom=="peanut" or geom=="Ushape" or geom=="circle":
        ncoef=CoefficientFunction([nval if mat=='D' else 1
                              for mat in mesh.GetMaterials()])
        scatterer=CoefficientFunction([3 if mat=="D" else 1 if mat=="air" else 2
                                for mat in mesh.GetMaterials()])       
    else:
        print('set ncoef for geom=',geom)
        zzz 
    Draw(scatterer,mesh,'scatterer')
    # solve the problem, plot results and save.
    uinf,theta,phi=helmsol(mesh,porder,ncoef,kappa,inc_p,far_p)
    plt.figure()
    plt.contourf(phi,theta,np.abs(uinf))
    plt.colorbar()
    plt.show()
    # Plot first far field pattern
    plt.figure()
    plt.plot(phi,np.real(uinf[:,0]))
    plt.plot(phi,np.imag(uinf[:,0]))
    plt.show()
    np.savez('farf.npz',kappa,uinf,theta,phi)